In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

In [2]:
smileon_full = pd.read_csv('../data2/SmileOn11.28.2020.csv', skiprows=2, encoding='ISO-8859-1')

C:\Users\kkosf\anaconda3\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (32,36,53,59,88,89,127,141,144,156,157,158,159) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# create a new dataframe with fewer columns:

smileon_full['CallDateAndTimeStart'] = pd.to_datetime(smileon_full['CallDateAndTimeStart'])
smileon_full['date'] = smileon_full['CallDateAndTimeStart'].dt.date
smileon_full['time'] = smileon_full['CallDateAndTimeStart'].dt.time

smileon = smileon_full[['CallReportNum', 'ReportVersion', 'date', 'CallerNum', 'CountyName', 'PostalCode', 
                        'SMILE ON 60+ Demographic Information - When is the last time you visited your dentist?',
                        'SMILE ON 60+ Case Management Information - What barriers have kept you from finding a consistent dental home? (Select all that apply)',
                        'SMILE ON 60+ Base-line Oral and Conditions Questions - Are you limited in what you can eat?',
                        'SMILE ON 60+ Base-line Oral and Conditions Questions - Have you ever felt that the appearance of your mouth and teeth affected your quality of life?',
                        'SMILE ON 60+ Base-line Oral and Conditions Questions - How would you describe the condition of your mouth and teeth?',
                        'SMILE ON 60+ Screening - How often do you brush your teeth?', 
                        'SMILE ON 60+ Screening - Need for Periodontal Care',
                        'SMILE ON 60+ Screening - Root Fragments', 
                        'SMILE ON 60+ Screening - Suspicious Soft Tissue Lesions',
                        'SMILE ON 60+ Screening - Untreated Decay', 
                        'SMILE ON 60+ Treatment Plan - Was a treatment plan developed?',
                        'SMILE ON 60+ Treatment Plan - Is the treatment plan completed?', 
                        'SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)'
                       ]]

#and rename the columns to make things easier:

smileon = smileon.rename(columns = {'SMILE ON 60+ Demographic Information - When is the last time you visited your dentist?': 'Last_Dentist_Visit',
                                    'SMILE ON 60+ Case Management Information - What barriers have kept you from finding a consistent dental home? (Select all that apply)' : 'Barriers',
                                    'SMILE ON 60+ Base-line Oral and Conditions Questions - Are you limited in what you can eat?' : 'Eating_Limitations',
                                    'SMILE ON 60+ Base-line Oral and Conditions Questions - Have you ever felt that the appearance of your mouth and teeth affected your quality of life?' : 'Appearance_QOL',
                                    'SMILE ON 60+ Base-line Oral and Conditions Questions - How would you describe the condition of your mouth and teeth?' : 'Condition',
                                    'SMILE ON 60+ Screening - How often do you brush your teeth?' : 'Brush_Frequency', 
                                    'SMILE ON 60+ Screening - Need for Periodontal Care' : 'Need_For_Care',
                                    'SMILE ON 60+ Screening - Root Fragments' : 'Root_Fragments', 
                                    'SMILE ON 60+ Screening - Suspicious Soft Tissue Lesions' : 'Lesions',
                                    'SMILE ON 60+ Screening - Untreated Decay' : 'Decay', 
                                    'SMILE ON 60+ Treatment Plan - Was a treatment plan developed?' : 'Tx_Plan_Developed',
                                    'SMILE ON 60+ Treatment Plan - Is the treatment plan completed?' : 'Tx_Plan_Complete', 
                                    'SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)' :
                                    'Function_Restored'
                                   })

smileon = smileon.loc[~smileon['CallerNum'].isin(['-1', '-2'])]

smileon.head()

,CallReportNum,ReportVersion,date,CallerNum,CountyName,PostalCode,Last_Dentist_Visit,Barriers,Eating_Limitations,Appearance_QOL,Condition,Brush_Frequency,Need_For_Care,Root_Fragments,Lesions,Decay,Tx_Plan_Developed,Tx_Plan_Complete,Function_Restored
0,81381827,SMILE ON 60+ Registration,2020-11-25,3859874,Robertson,37172.0,3 to 5 years,Transportation; Dental cost; No insurance,No,Yes,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81379177,SMILE ON 60+ Registration,2020-11-25,3859755,Davidson,37212.0,3 to 5 years,Dental cost; No insurance,No,No,Very Good,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,81377435,SMILE ON 60+ Clinical,2020-11-25,2518680,Hawkins,37857.0,Less than 12 months,Dental cost,No,No,Good,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes
3,81375215,SMILE ON 60+ Clinical,2020-11-25,3231923,Shelby,38104.0,Less than 12 months,Transportation; Dental cost; No insurance,Yes,Yes,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,81371535,SMILE ON 60+ Clinical,2020-11-25,3605119,Sullivan,37664.0,Less than 12 months,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,No,NaN


In [4]:
smileon.isnull().sum()

CallReportNum             0
ReportVersion             0
date                      0
CallerNum                 0
CountyName              314
PostalCode             2569
Last_Dentist_Visit     2740
Barriers               5350
Eating_Limitations     6206
Appearance_QOL         6730
Condition              6165
Brush_Frequency       35388
Need_For_Care         35524
Root_Fragments        35296
Lesions               35590
Decay                 35130
Tx_Plan_Developed     22930
Tx_Plan_Complete      22418
Function_Restored     25680
dtype: int64

In [5]:
smileon.CallerNum.nunique()

8706

In [6]:
Plan_developed_all = smileon[smileon['Tx_Plan_Developed'] == 'Yes'] 
Plan_developed_all.CallerNum.nunique()

4814

In [7]:
smileon.ReportVersion.value_counts()

SMILE ON 60+ Clinical        27357
SMILE ON 60+ Registration    12595
Name: ReportVersion, dtype: int64

In [8]:
#separate out registration vs clinical:

reg_only = smileon[smileon['ReportVersion'] == 'SMILE ON 60+ Registration'] 
print(reg_only.shape)

clinic_only =  smileon[smileon['ReportVersion'] == 'SMILE ON 60+ Clinical'] 
print(clinic_only.shape)

(12595, 19)
(27357, 19)


In [9]:
clinic_only.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27357 entries, 2 to 40029
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CallReportNum       27357 non-null  int64  
 1   ReportVersion       27357 non-null  object 
 2   date                27357 non-null  object 
 3   CallerNum           27357 non-null  int64  
 4   CountyName          27177 non-null  object 
 5   PostalCode          25680 non-null  float64
 6   Last_Dentist_Visit  25527 non-null  object 
 7   Barriers            23954 non-null  object 
 8   Eating_Limitations  23237 non-null  object 
 9   Appearance_QOL      22898 non-null  object 
 10  Condition           23244 non-null  object 
 11  Brush_Frequency     4564 non-null   object 
 12  Need_For_Care       4428 non-null   object 
 13  Root_Fragments      4656 non-null   object 
 14  Lesions             4362 non-null   object 
 15  Decay               4822 non-null   object 
 16  Tx_P

In [10]:
clinic_only.CallerNum.nunique()

5417

In [11]:
reg_only.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12595 entries, 0 to 40028
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CallReportNum       12595 non-null  int64  
 1   ReportVersion       12595 non-null  object 
 2   date                12595 non-null  object 
 3   CallerNum           12595 non-null  int64  
 4   CountyName          12461 non-null  object 
 5   PostalCode          11703 non-null  float64
 6   Last_Dentist_Visit  11685 non-null  object 
 7   Barriers            10648 non-null  object 
 8   Eating_Limitations  10509 non-null  object 
 9   Appearance_QOL      10324 non-null  object 
 10  Condition           10543 non-null  object 
 11  Brush_Frequency     0 non-null      object 
 12  Need_For_Care       0 non-null      object 
 13  Root_Fragments      0 non-null      object 
 14  Lesions             0 non-null      object 
 15  Decay               0 non-null      object 
 16  Tx_P

In [12]:
#using the whole dataset, the number of patients with funtion restored: 3356

Function_restored = smileon[smileon['Function_Restored'] == 'Yes'] 
print(Function_restored.CallerNum.nunique())

3356/8706 #39% of all patients have function restored

3356


0.3854812772800368

In [13]:
#using the clinic only data, the number of patients with a treatment plan developed: 4814

Plan_developed = clinic_only[clinic_only['Tx_Plan_Developed'] == 'Yes'] 
Plan_developed.CallerNum.nunique()

4814

In [14]:
#Those with a treatment plan developed that were restored to function: 3116

Plan_developed_fx = Plan_developed[Plan_developed['Function_Restored'] == 'Yes'] 
print(Plan_developed_fx.CallerNum.nunique())
3116/4814 #65% of those with a treatment plan had function restored

3116


0.6472787702534275

In [15]:
#subset those that reported eating limitations that had a treatment plan developed: 2220

Eating_Limitations = Plan_developed[Plan_developed['Eating_Limitations'] == 'Yes'] 
Eating_Limitations.CallerNum.nunique()

2220

In [16]:
#Those with a treatment plan developed and reported eating limitations that were restored to function: 1298

Eating_Limitations_fx = Eating_Limitations[Eating_Limitations['Function_Restored'] == 'Yes'] 
print(Eating_Limitations_fx.CallerNum.nunique())

1298/2220 #58% of people that reported eating limitations have had function restored

1298


0.5846846846846847

In [17]:
#subset those that reported appearance and quality of life issues that had a treatment plan developed: 2661

Appearance_QOL = Plan_developed[Plan_developed['Appearance_QOL'] == 'Yes'] 
Appearance_QOL.CallerNum.nunique()

2661

In [18]:
#Those with a treatment plan developed and reported appearance and quality of life issues that were restored to function: 1614

Appearance_QOL_fx = Appearance_QOL[Appearance_QOL['Function_Restored'] == 'Yes'] 
print(Appearance_QOL_fx.CallerNum.nunique())
1614/2661 #61% of people that reported appearance/QOL issues have had function restored

1614


0.6065388951521984

In [19]:
#Checking options for mouth condition:

Plan_developed.Condition.value_counts()

Poor         5967
Fair         4799
Good         1908
Dont know     598
Very Good     243
Name: Condition, dtype: int64

In [20]:
#subset those that reported poor mouth condition that had a treatment plan developed: 1741
Poor_condition = Plan_developed[Plan_developed['Condition'] == 'Poor']
Poor_condition.CallerNum.nunique()

1741

In [21]:
#Those with a treatment plan developed and reported poor mouth condition that were restored to function: 979

Poor_condition_fx = Poor_condition[Poor_condition['Function_Restored'] == 'Yes'] 
print(Poor_condition_fx.CallerNum.nunique())
979/1741 #56% of people that reported poor mouth condition have had function restored

979


0.5623205054566341

In [22]:
#subset those that reported fair mouth condition that had a treatment plan developed: 1475
Fair_condition = Plan_developed[Plan_developed['Condition'] == 'Fair']
Fair_condition.CallerNum.nunique()

1475

In [23]:
#Those with a treatment plan developed and reported poor mouth condition that were restored to function: 969

Fair_condition_fx = Fair_condition[Fair_condition['Function_Restored'] == 'Yes'] 
print(Fair_condition_fx.CallerNum.nunique())
969/1475 #66% of people that reported fair mouth condition have had function restored

969


0.6569491525423728

In [24]:
Plan_developed.Last_Dentist_Visit.value_counts()

Less than 12 months    7290
More than 5 years      2888
1 to 2 years           2711
3 to 5 years           1806
Name: Last_Dentist_Visit, dtype: int64

In [25]:
#subset those that have not been to the dentist in more than 5 years that had a treatment plan developed: 947
five_plus = Plan_developed[Plan_developed['Last_Dentist_Visit'] == 'More than 5 years'] 
five_plus.CallerNum.nunique()

947

In [26]:
#Those with a treatment plan developed that have not been to the dentist in more than 5 years that were restored to function: 516

five_plus_fx = five_plus[five_plus['Function_Restored'] == 'Yes'] 
print(five_plus_fx.CallerNum.nunique())
516/947 #54% of people that haven't been to the dentist in 5 years have function restored

516


0.5448785638859557

In [27]:
Plan_developed.Decay.value_counts()

# 'SMILE ON 60+ Screening - Need for Periodontal Care' : 'Need_For_Care',
# 'SMILE ON 60+ Screening - Root Fragments' : 'Root_Fragments', 
# 'SMILE ON 60+ Screening - Suspicious Soft Tissue Lesions' : 'Lesions',
# 'SMILE ON 60+ Screening - Untreated Decay' : 'Decay', 

Yes           2532
No             998
Edentulous     165
Name: Decay, dtype: int64

In [28]:
#subset those that have 'Need for periodontal Care' that had a treatment plan developed: 1471

Need_For_Care = Plan_developed[Plan_developed['Need_For_Care'] == 'Yes'] 

Need_For_Care.CallerNum.nunique()

1471

In [29]:
#Those with a treatment plan developed that have 'Need for periodontal Care' that were restored to function: 319

Need_For_Care_fx = Need_For_Care[Need_For_Care['Function_Restored'] == 'Yes'] 
print(Need_For_Care_fx.CallerNum.nunique())
319/1471 #22% of people that have 'Need for periodontal Care' have function restored

319


0.2168592794017675

In [30]:
#subset those that have 'Root Fragments' that had a treatment plan developed: 940

Root_Fragments = Plan_developed[Plan_developed['Root_Fragments'] == 'Yes'] 

Root_Fragments.CallerNum.nunique()

940

In [31]:
#Those with a treatment plan developed that have 'Root Fragments' that were restored to function: 147

Root_Fragments_fx = Root_Fragments[Root_Fragments['Function_Restored'] == 'Yes'] 
print(Root_Fragments_fx.CallerNum.nunique())
147/940 #16% of people that have 'Root Fragments' have function restored

147


0.15638297872340426

In [32]:
#subset those that have 'Suspicious Soft Tissue Lesions' that had a treatment plan developed: 150

Lesions = Plan_developed[Plan_developed['Lesions'] == 'Yes'] 

Lesions.CallerNum.nunique()

150

In [33]:
#Those with a treatment plan developed that have 'Suspicious Soft Tissue Lesions' that were restored to function: 33

Lesions_fx = Lesions[Lesions['Function_Restored'] == 'Yes'] 
print(Lesions_fx.CallerNum.nunique())
33/150 #22% of people that have 'Suspicious Soft Tissue Lesions' have function restored

33


0.22

In [34]:
#subset those that have 'Untreated Decay' that had a treatment plan developed: 2298

Decay = Plan_developed[Plan_developed['Decay'] == 'Yes'] 

Decay.CallerNum.nunique()

2298

In [35]:
#Those with a treatment plan developed that have 'Untreated Decay' that were restored to function: 475

Decay_fx = Decay[Decay['Function_Restored'] == 'Yes'] 
print(Decay_fx.CallerNum.nunique())
475/2298 #20% of people that have 'Untreated Decay' have function restored

475


0.20670147954743254

In [36]:
Decay_all = clinic_only[clinic_only['Decay'] == 'Yes'] 

Decay_all.CallerNum.nunique()

2894

In [37]:
Decay_fx_all = Decay_all[Decay_all['Function_Restored'] == 'Yes'] 
print(Decay_fx_all.CallerNum.nunique())

511


In [38]:
511/2894

0.17657221838286108

In [39]:
encounters_per_pt = smileon.CallerNum.value_counts()
encounters_per_pt

2441776    36
2431132    31
2416076    29
2860770    28
2466994    28
           ..
2536760     1
2510267     1
3200276     1
2450144     1
2496365     1
Name: CallerNum, Length: 8706, dtype: int64

In [40]:
clinic_encounters_per_pt = clinic_only.CallerNum.value_counts()
clinic_encounters_per_pt.to_csv(r'C:\Users\kkosf\Documents\nss\projects\smile-on-spiderman\clinic_encounters_per_pt.csv', index=False)
clinic_encounters_per_pt

2441776    35
2431132    30
2416076    28
2860770    27
2466994    27
           ..
3464703     1
3165725     1
2758182     1
2520618     1
3041413     1
Name: CallerNum, Length: 5417, dtype: int64